In [1]:
import os
import pandas as pd
import sys
from datetime import datetime, timedelta
from pandas.tseries.offsets import BDay

# Caminhos dos seus módulos customizados
sys.path.insert(0, r'C:\Scripts\modules\database')
sys.path.insert(0, r'C:\Scripts\modules\parameters')

from connection import Connect
from bases import Bases
from parametros import Parametros

diretorio_base = r"C:\Scripts\relatórios"

In [2]:
# %%
pastas_dias = []

# Coleta as pastas diretamente na raiz de relatórios (pulando a pasta mês)
for dia in next(os.walk(r"C:\\Scripts\\relatórios"))[1]:
    try:
        datetime.strptime(dia, "%d-%m-%Y")
        pastas_dias.append({"dia": dia})
    except ValueError:
        continue

# Ordena igual ao original
pastas_dias.sort(key = lambda date: datetime.strptime(date["dia"], "%d-%m-%Y"))

# Lista para armazenar os DataFrames de cada dia
lista_entrada_saida = []

for presente in pastas_dias:
    dia_str = presente['dia']
    diretorio_pastas_presente = os.path.join(diretorio_base, dia_str)
    
    try:
        subpastas = os.listdir(diretorio_pastas_presente)
        if not subpastas:
            continue
            
        base_hoje = subpastas[0]
        caminho_final = os.path.join(diretorio_pastas_presente, base_hoje, "base_btg.xlsx")
        
        # Tenta ler o arquivo
        try:
            df = pd.read_excel(caminho_final, header=2)
            if 'Conta' not in df.columns:
                df = pd.read_excel(caminho_final)
        except:
            df = pd.read_excel(caminho_final)

        # Padroniza colunas (Remove " (R$)" e espaços)
        df.columns = [str(c).replace(" (R$)", "").strip() for c in df.columns]

        # Mostra o dia e as colunas originais encontradas antes de renomear
        print(f"--- Processando dia: {dia_str} ---")
        print(f"Colunas encontradas: {list(df.columns)}")

        # Tratamento da coluna de abertura
        mapeamento = {'Data de Abertura da Conta': 'Data de Abertura', 'Data Vínculo Assessor': 'Data Vínculo'}
        df.rename(columns=mapeamento, inplace=True)

        # Seleção das colunas
        colunas_alvo = ['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Faixa Cliente', 'PL Total']
        colunas_existentes = [c for c in colunas_alvo if c in df.columns]
        
        # DEBUG: Confirma o que foi selecionado após o tratamento
        print(f"Colunas selecionadas: {colunas_existentes}\n")

        df_final = df[colunas_existentes].copy()
        
        if 'Conta' in df_final.columns:
            df_final['Conta'] = df_final['Conta'].astype(str)

        lista_entrada_saida.append(df_final)

    except Exception as e:
        print(f"!!! ERRO no dia {dia_str}: {e}\n")

historico_abertura = pd.concat(lista_entrada_saida, ignore_index=True)

--- Processando dia: 27-11-2025 ---
Colunas encontradas: ['Conta', 'Nome', 'Assessor', 'Tipo Parceiro', 'E-Mail Comunicação', 'Carteira Administrada', 'Tipo', 'Aniversário', 'Profissão / Setor', 'Estado Civil', 'Cidade', 'Estado', 'Data de Abertura do Assessor', 'Data Vínculo', 'Tipo Investidor', 'Termo de Marcação na Curva', 'Faixa Cliente', '1º Aporte', 'Último Aporte', 'Qtd de Aportes', 'Aportes', 'Retiradas', 'Qtd de Ativos', 'Qtd Fundos', 'Qtd Renda Fixa', 'Qtd Renda Variável', 'Qtd Previdência', 'Qtd Derivativos', 'Qtd Valor em Trânsito', 'PL Total', 'Conta Corrente', 'Fundos', 'Renda Fixa', 'Renda Variável', 'Previdência', 'Derivativos', 'Valor em Trânsito', 'Renda Anual', 'PL Declarado', 'Código do Escritório', 'Escritório', 'Código do Assessor', 'Data Vínculo Escritório', 'E-mail Assessor', 'ID Cliente']
Colunas selecionadas: ['Conta', 'Assessor', 'Data Vínculo', 'Faixa Cliente', 'PL Total']

--- Processando dia: 04-12-2025 ---
Colunas encontradas: ['Conta', 'Nome', 'Assessor'

In [3]:
lista_entrada_saida = []
diretorio_base = r"C:\Scripts\relatórios"

pastas_processadas = 0
datas_lidas = []

print("Iniciando processamento...\n")

for presente in pastas_dias:
    dia_str = presente['dia']
    caminho_dia = os.path.join(diretorio_base, dia_str)
    
    try:
        subpastas = os.listdir(caminho_dia)
        if not subpastas:
            continue
            
        pasta_hora = subpastas[0]
        caminho_arquivo = os.path.join(caminho_dia, pasta_hora, "base_btg.xlsx")

        # Leitura do Excel
        try:
            df = pd.read_excel(caminho_arquivo, header=2)
            if 'Conta' not in df.columns:
                df = pd.read_excel(caminho_arquivo)
        except:
            df = pd.read_excel(caminho_arquivo)

        # Padronização e Mapeamento
        df.columns = [str(c).replace(" (R$)", "").strip() for c in df.columns]
        
        mapeamento = {
            'Data de Abertura da Conta': 'Data de Abertura',
            'Data de Abertura do Assessor': 'Data de Abertura',
            'Data Vínculo Assessor': 'Data Vínculo'
        }
        df.rename(columns=mapeamento, inplace=True)

        colunas_alvo = ['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Faixa Cliente', 'PL Total']
        colunas_final = [c for c in colunas_alvo if c in df.columns]
        
        base_atual = df[colunas_final].copy()
        if 'Conta' in base_atual.columns:
            base_atual['Conta'] = base_atual['Conta'].astype(str)

        lista_entrada_saida.append(base_atual)
        
        # Atualiza contadores para o Debug
        pastas_processadas += 1
        datas_lidas.append(datetime.strptime(dia_str, "%d-%m-%Y"))

        # Log individual simplificado para não poluir o console
        print(f"[OK] {dia_str} | Colunas: {len(colunas_final)}")

    except Exception as e:
        print(f"[ERRO] Dia {dia_str}: {e}")

# --- RESUMO DO DEBUG ---
print("\n" + "="*30)
print("      RESUMO DO PROCESSO")
print("="*30)
if datas_lidas:
    print(f"Total de pastas lidas: {pastas_processadas}")
    print(f"Data inicial: {min(datas_lidas).strftime('%d/%m/%Y')}")
    print(f"Data final:   {max(datas_lidas).strftime('%d/%m/%Y')}")
    print(f"Última pasta lida: {dia_str}")
else:
    print("Nenhuma pasta foi processada.")
print("="*30 + "\n")

if lista_entrada_saida:
    historico_abertura = pd.concat(lista_entrada_saida, ignore_index=True)

Iniciando processamento...

[OK] 27-11-2025 | Colunas: 6
[OK] 04-12-2025 | Colunas: 6
[OK] 05-12-2025 | Colunas: 6
[OK] 08-12-2025 | Colunas: 6
[OK] 09-12-2025 | Colunas: 6
[OK] 10-12-2025 | Colunas: 6
[OK] 11-12-2025 | Colunas: 6
[OK] 12-12-2025 | Colunas: 6
[OK] 15-12-2025 | Colunas: 6
[OK] 16-12-2025 | Colunas: 6
[OK] 17-12-2025 | Colunas: 6
[OK] 18-12-2025 | Colunas: 6
[OK] 05-01-2026 | Colunas: 6
[OK] 06-01-2026 | Colunas: 6
[OK] 07-01-2026 | Colunas: 6
[OK] 08-01-2026 | Colunas: 6
[OK] 09-01-2026 | Colunas: 6
[OK] 12-01-2026 | Colunas: 6
[OK] 13-01-2026 | Colunas: 6
[OK] 14-01-2026 | Colunas: 6
[OK] 15-01-2026 | Colunas: 6
[OK] 16-01-2026 | Colunas: 6
[OK] 19-01-2026 | Colunas: 6
[OK] 20-01-2026 | Colunas: 6
[OK] 21-01-2026 | Colunas: 6
[OK] 22-01-2026 | Colunas: 6
[OK] 23-01-2026 | Colunas: 6
[OK] 26-01-2026 | Colunas: 6
[OK] 27-01-2026 | Colunas: 6
[OK] 28-01-2026 | Colunas: 6
[OK] 29-01-2026 | Colunas: 6

      RESUMO DO PROCESSO
Total de pastas lidas: 31
Data inicial: 27/11/2

In [4]:
historico_abertura = pd.concat(lista_entrada_saida)

In [5]:
historico_abertura = \
    historico_abertura[
        ~((historico_abertura['Assessor'] == 'Gabriel Rodrigues') &
        (historico_abertura['Data Vínculo'] >= "2024-02-01") &
        (historico_abertura['Data Vínculo'] <= "2024-03-01"))
    ]
    
historico_abertura = \
    historico_abertura[
        ~((historico_abertura['Assessor'] == 'Vinicius Servino Vargas') &
        (historico_abertura['Data Vínculo'] == "2024-02-08"))
    ]

In [6]:
conn = Connect.connect_techdb()
entradas_saidas_consolidado = Connect.import_table(conn, 'Entradas_e_saidas_consolidado')
conn.close()

In [7]:
entradas_saidas_consolidado

,Conta,Assessor,PL Total,PL Declarado,Faixa Cliente,Data Vínculo,Situação,Mês de entrada/saída,Nome
0,1987215,FERNANDO DOMINGUES DA SILVA,6.500000e-01,660000.0,Ate 50K,2021-07-12,Saiu,2021-10-18,BRUNO VINICIUS SANTOS
1,2589680,VICTOR ALMEIDA BERNASCONI,NaN,NaN,None,2021-10-18,Entrou,2021-10-18,GUILHERME KOCH LERNER
2,2369171,ROSANA APARECIDA PAVANI DA SILVA,1.329697e+06,25000000.0,None,2021-10-18,Entrou,2021-10-18,BRUNO DARIO WERNECK
3,1718366,CARLOS FERNANDO PERICO,8.768894e+05,9250000.0,Entre 500k e 1MM,2021-07-05,Saiu,2021-10-19,ROGERIO MIGUEL PEREZ
4,1399004,FERNANDO DOMINGUES DA SILVA,1.530000e+00,3100000.0,Ate 50K,2021-07-09,Saiu,2021-10-19,BRUNA DREON GOMES CORREA DO NASCIMENTO
...,...,...,...,...,...,...,...,...,...
5447,17593583,CAIC ZEM GOMES,5.150000e+02,1420000.0,None,NaT,Entrou,2026-01-29,Davi
5448,18774067,PAULO ROBERTO FARIA SILVA,2.030286e+06,10000000.0,None,NaT,Entrou,2026-01-29,Dino
5449,21236666,PAULO ROBERTO FARIA SILVA,NaN,1000000.0,None,NaT,Entrou,2026-01-29,Formenton
5450,21246129,CAIC ZEM GOMES,NaN,4000000.0,None,NaT,Entrou,2026-01-29,Marineia


In [8]:
sairam_entraram = historico_abertura.merge(entradas_saidas_consolidado[entradas_saidas_consolidado['Situação'] == 'Saiu'][['Conta', 'Mês de entrada/saída']], on='Conta', how='left')
sairam_entraram = \
    sairam_entraram[
        (sairam_entraram['Mês de entrada/saída'] < sairam_entraram['Data Vínculo'])
    ]
sairam_entraram.drop("Mês de entrada/saída", axis=1, inplace=True)

In [9]:
historico_abertura = historico_abertura[~historico_abertura['Conta'].isin(sairam_entraram['Conta'])]
historico_abertura = pd.concat([historico_abertura, sairam_entraram], axis=0)

In [10]:
historico_abertura.drop_duplicates(subset=["Conta"], keep='first', inplace=True)
historico_abertura.rename(columns={"Faixa Cliente":"Faixa Cliente Abertura", "PL Total":"PL Abertura"}, inplace=True)

In [11]:
primeiro_vinculo = historico_abertura[['Assessor', 'Conta', 'Data Vínculo', 'Faixa Cliente Abertura', 'PL Abertura']]
#primeiro_vinculo.rename(columns={"Data Vínculo":"Primeiro Vínculo"}, inplace=True)

In [12]:
conexao = Connect.connect_techdb()
basebtg = Connect.import_table(conexao, "base_btg")
conexao.close()

# 1. Padroniza os nomes das colunas (remove espaços extras que o banco pode trazer)
basebtg.columns = basebtg.columns.str.strip()

# 2. Mapeia todas as versões possíveis da coluna de abertura
# Se o banco trouxer o nome novo, ele converte corretamente
mapeamento_abertura = {
    'Data de Abertura do Assessor': 'Data de Abertura',
    'Data de Abertura da Conta': 'Data de Abertura'
}
basebtg.rename(columns=mapeamento_abertura, inplace=True)

# 3. Faz o merge apenas com as colunas que agora temos certeza que existem
try:
    base_vinculos = primeiro_vinculo.merge(
        basebtg[['Conta', 'Data de Abertura']], 
        on='Conta', 
        how='left'
    )
    base_vinculos = base_vinculos.dropna(subset="Data Vínculo")
    print("Merge realizado com sucesso usando a coluna: Data de Abertura" )
except KeyError:
    # Caso o banco tenha um nome totalmente diferente, esse debug te avisará qual é
    print(f"Erro: Não achei a coluna de abertura. Colunas no banco são: {basebtg.columns.tolist()}")

Merge realizado com sucesso usando a coluna: Data de Abertura


In [13]:
base_vinculos = primeiro_vinculo.merge(basebtg[['Conta', 'Data de Abertura']], on='Conta', how='left')
base_vinculos = base_vinculos.dropna(subset="Data Vínculo")

In [14]:
base_filtrada = base_vinculos.copy()

In [15]:
origem = pd.read_excel(r'C:\Scripts\backups_atria\historico_relatorios\historico_nnm\NNM Válido Potenza - Parcial 07.02.2023.xlsx')
origem = origem[['Conta', 'Origem da Conta']]
origem['Conta'] = origem['Conta'].astype(str)
origem.drop_duplicates(subset="Conta", inplace=True)

In [16]:
base_filtrada = base_filtrada.merge(origem, on='Conta', how='left')

In [17]:
base_filtrada = base_filtrada[[
    'Conta', 'Assessor','Data de Abertura', 'Data Vínculo', 
    'Faixa Cliente Abertura', 'Origem da Conta', 'PL Abertura'
    ]]

In [18]:
base_filtrada['Tipo'] = ''

In [19]:
base_filtrada.loc[
    (base_filtrada['Data de Abertura'] == base_filtrada['Data Vínculo']),
    "Tipo"
] = "Abertura"

In [20]:
base_filtrada.loc[
    (base_filtrada['Data Vínculo'] != base_filtrada['Data de Abertura']),
    "Tipo"
] = "Migração"

In [21]:
base_filtrada = base_filtrada[~base_filtrada['Data de Abertura'].isna()]

In [22]:
base_filtrada['Mes Abertura'] = [data.strftime("%Y-%m") for data in base_filtrada['Data de Abertura']]
base_filtrada['Mes Vinculo'] = [data.strftime("%Y-%m") for data in base_filtrada['Data Vínculo']]
base_filtrada['PL Abertura'].fillna(0, inplace=True)

C:\Users\joaob\AppData\Local\Temp\ipykernel_4244\3135824580.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base_filtrada['PL Abertura'].fillna(0, inplace=True)


In [23]:
base_filtrada = base_filtrada[
                    (base_filtrada['Tipo'] != "Migração Interna") &
                    (base_filtrada['Tipo'] != "")
                    ]

In [24]:
base_filtrada = pd.merge(base_filtrada, basebtg[['Conta', 'PL Total']], on='Conta', how='left')
base_filtrada['PL Total'].fillna(0, inplace=True)
base_filtrada.rename(columns={"PL Total":"PL Atual"}, inplace=True)

C:\Users\joaob\AppData\Local\Temp\ipykernel_4244\2040650073.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  base_filtrada['PL Total'].fillna(0, inplace=True)


In [25]:
conexao = Connect.connect_techdb()
pl_historico = Connect.import_table(conexao, "PL Base")
conexao.close()

In [26]:
contas_pl = base_filtrada['Conta']

for conta in contas_pl:
    try:
        if len(pl_historico[(pl_historico['CONTA'] == conta)].sort_values("Mês")) > 1:
            pl_mes_abertura = pl_historico[(pl_historico['CONTA'] == conta)].sort_values("Mês").iloc[1,:]['PL']
            base_filtrada.loc[(base_filtrada['Conta'] == conta), 'PL Abertura'] = pl_mes_abertura
        else:
            pl_mes_abertura = pl_historico[(pl_historico['CONTA'] == conta)].sort_values("Mês")['PL'].values[0]
            base_filtrada.loc[(base_filtrada['Conta'] == conta), 'PL Abertura'] = pl_mes_abertura
    except:
        pass

In [27]:
base_filtrada.loc[base_filtrada['PL Abertura'] < 500000, "Faixa Cliente Abertura"] = "< 500k"
base_filtrada.loc[(base_filtrada['PL Abertura'] >= 500000) & (base_filtrada['PL Abertura'] < 1000000), "Faixa Cliente Abertura"] = "> 500k e < 1mm"
base_filtrada.loc[(base_filtrada['PL Abertura'] >= 1000000) & (base_filtrada['PL Abertura'] < 5000000), "Faixa Cliente Abertura"] = "> 1mm e < 5mm"
base_filtrada.loc[base_filtrada['PL Abertura'] > 5000000, "Faixa Cliente Abertura"] = "> 5mm"

In [28]:
base_filtrada.loc[base_filtrada['PL Atual'] < 500000, "Faixa Cliente Atual"] = "< 500k"
base_filtrada.loc[(base_filtrada['PL Atual'] >= 500000) & (base_filtrada['PL Atual'] < 1000000), "Faixa Cliente Atual"] = "> 500k e < 1mm"
base_filtrada.loc[(base_filtrada['PL Atual'] >= 1000000) & (base_filtrada['PL Atual'] < 5000000), "Faixa Cliente Atual"] = "> 1mm e < 5mm"
base_filtrada.loc[base_filtrada['PL Atual'] > 5000000, "Faixa Cliente Atual"] = "> 5mm"

In [29]:
base_filtrada.loc[base_filtrada['PL Atual'] < 1000000, "Faixa Cliente Performance"] = "< 1MM"
base_filtrada.loc[base_filtrada['PL Atual'] >= 1000000, "Faixa Cliente Performance"] = "> 1MM"

In [30]:
base_filtrada.loc[base_filtrada['Assessor'] == "Rodrigo de Mello D?Elia", "Assessor"] = "Rodrigo de Mello D’Elia"
base_filtrada.loc[base_filtrada['Assessor'] == "Rodrigo de Mello DElia", "Assessor"] = "Rodrigo de Mello D’Elia"

In [31]:
base_filtrada['Assessor'] = [nome.upper() for nome in base_filtrada['Assessor']]

In [32]:
tempo_potenza = base_filtrada[['Conta', 'Data Vínculo', 'Assessor']]

In [33]:
tempo_potenza

,Conta,Data Vínculo,Assessor
0,246022,2025-07-01,ROSANA APARECIDA PAVANI DA SILVA
1,264671,2025-07-01,FERNANDO DOMINGUES DA SILVA
2,290492,2025-07-01,ROSANA APARECIDA PAVANI DA SILVA
3,299305,2025-11-19,GABRIEL GUERRERO TORRES FONSECA
4,321039,2025-07-01,CAIC ZEM GOMES
...,...,...,...
1186,8565242,2025-12-15,FERNANDO DOMINGUES DA SILVA
1187,2176947,2025-12-19,GUILHERME DE LUCCA BERTELONI
1188,4409896,2026-01-07,GABRIEL GUERRERO TORRES FONSECA
1189,5496682,2026-01-09,GABRIEL GUERRERO TORRES FONSECA


In [34]:
conexao = Connect.connect_techdb()
tempo_potenza.to_sql("primeiro_vinculo", 
                     con=conexao, 
                     index=False, 
                     if_exists='replace',
                     schema='principal')
conexao.close()

In [35]:
base_filtrada = base_filtrada[
        (base_filtrada['Data Vínculo'] >= "2022-01-01") |
        (base_filtrada['Data de Abertura'] >= "2022-01-01")
    ]

In [36]:
nomes_clientes = pd.read_excel(r"C:\\Scripts\\nomes_clientes\\Nomes_Clientes2.xlsx")

In [37]:
base_filtrada = base_filtrada.merge(nomes_clientes, on='Conta', how='left')

In [38]:
base_filtrada

,Conta,Assessor,Data de Abertura,Data Vínculo,Faixa Cliente Abertura,Origem da Conta,PL Abertura,Tipo,Mes Abertura,Mes Vinculo,PL Atual,Faixa Cliente Atual,Faixa Cliente Performance,Nome
0,246022,ROSANA APARECIDA PAVANI DA SILVA,2016-03-24,2025-07-01,> 1mm e < 5mm,B2C,1207795.26,Migração,2016-03,2025-07,1890761.76,> 1mm e < 5mm,> 1MM,ELIETE DE CARVALHO FRADE
1,264671,FERNANDO DOMINGUES DA SILVA,2016-08-01,2025-07-01,> 1mm e < 5mm,ADVISORS,3864315.74,Migração,2016-08,2025-07,2760779.44,> 1mm e < 5mm,> 1MM,FERNANDO DOMINGUES DA SILVA
2,290492,ROSANA APARECIDA PAVANI DA SILVA,2017-01-11,2025-07-01,< 500k,B2C,22204.83,Migração,2017-01,2025-07,104116.29,< 500k,< 1MM,DIEGO DE CARVALHO FRADE
3,299305,GABRIEL GUERRERO TORRES FONSECA,2017-02-23,2025-11-19,> 500k e < 1mm,NaN,994315.53,Migração,2017-02,2025-11,157080.84,< 500k,< 1MM,KELLY BOEGER KEINER
4,321039,CAIC ZEM GOMES,2017-05-04,2025-07-01,< 500k,B2C,204369.92,Migração,2017-05,2025-07,0.63,< 500k,< 1MM,LUCAS MATT CURY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1186,8565242,FERNANDO DOMINGUES DA SILVA,2025-03-26,2025-12-15,> 500k e < 1mm,NaN,806924.37,Migração,2025-03,2025-12,608130.56,> 500k e < 1mm,< 1MM,Luizvictor
1187,2176947,GUILHERME DE LUCCA BERTELONI,2021-07-19,2025-12-19,< 500k,NaN,123658.08,Migração,2021-07,2025-12,56090.93,< 500k,< 1MM,Henrique
1188,4409896,GABRIEL GUERRERO TORRES FONSECA,2023-02-21,2026-01-07,< 500k,NaN,0.00,Migração,2023-02,2026-01,2706547.50,> 1mm e < 5mm,> 1MM,RAPHAEL FERREIRA ROCHA
1189,5496682,GABRIEL GUERRERO TORRES FONSECA,2020-11-12,2026-01-09,< 500k,-,32.00,Migração,2020-11,2026-01,906.79,< 500k,< 1MM,MARCOS SOARES PEREIRA FILHO


In [39]:
base_filtrada.loc[base_filtrada['Conta'] == '590732', "Assessor"] = "JOSE AUGUSTO ALVES DE PAULA FILHO"
base_filtrada.loc[base_filtrada['Conta'] == '299305', "Assessor"] = "JOSE AUGUSTO ALVES DE PAULA FILHO"

In [40]:
# %%
# Ordena pela Data Vínculo do mais recente para o mais antigo
base_recente = base_filtrada.sort_values(by='Data Vínculo', ascending=False)

print(f"--- Top 10 registros com as datas de vínculo mais recentes ---")
print(f"Total de linhas na base: {len(base_recente)}")

# Mostra as colunas principais para facilitar a conferência
display(base_recente[['Conta', 'Assessor', 'Data de Abertura', 'Data Vínculo', 'Tipo', 'PL Atual']].head(10))

# %%
# DEBUG: Verificando se existem datas nulas que podem estar fugindo da ordenação
nulos = base_recente['Data Vínculo'].isna().sum()
if nulos > 0:
    print(f"Atenção: Existem {nulos} linhas com Data Vínculo vazia.")

--- Top 10 registros com as datas de vínculo mais recentes ---
Total de linhas na base: 1191


,Conta,Assessor,Data de Abertura,Data Vínculo,Tipo,PL Atual
1154,21312293,RODRIGO DE MELLO D’ELIA,2026-01-28,2026-01-28,Abertura,0.00
1150,17593583,CAIC ZEM GOMES,2026-01-27,2026-01-27,Abertura,515.00
1149,16458051,RENAN BENTO DA SILVA,2026-01-27,2026-01-27,Abertura,0.00
1153,21246129,CAIC ZEM GOMES,2026-01-27,2026-01-27,Abertura,0.00
1151,18774067,PAULO ROBERTO FARIA SILVA,2026-01-27,2026-01-27,Abertura,2030285.94
1152,21236666,PAULO ROBERTO FARIA SILVA,2026-01-27,2026-01-27,Abertura,0.00
1148,20990156,GUILHERME DE LUCCA BERTELONI,2026-01-22,2026-01-22,Abertura,70099.61
1147,20904405,FELIPE AUGUSTO CARDOSO,2026-01-22,2026-01-22,Abertura,50095.37
1146,20925566,CAIC ZEM GOMES,2026-01-21,2026-01-21,Abertura,0.00
1145,2667332,CAIC ZEM GOMES,2021-11-19,2026-01-21,Migração,291763.26


In [41]:
conexao = Connect.connect_techdb()
base_filtrada.to_sql("abertura_de_conta_base", 
                     con=conexao, 
                     index=False, 
                     if_exists='replace',
                     schema='principal')
conexao.close()

In [42]:
abertura = base_filtrada[base_filtrada['Tipo'] == "Abertura"]
abertura = abertura.groupby(["Assessor", "Mes Abertura", "Faixa Cliente Abertura", "Faixa Cliente Atual", "Faixa Cliente Performance"])['Data de Abertura'].count().reset_index()

In [43]:
vinculo = base_filtrada[base_filtrada['Tipo'] == "Migração"]
vinculo = vinculo.groupby(["Assessor", "Mes Vinculo", "Faixa Cliente Abertura", "Faixa Cliente Atual", "Faixa Cliente Performance"])['Data Vínculo'].count().reset_index()

In [44]:
abertura.rename(columns={"Mes Abertura":"Mes", "Data de Abertura":"Contas Abertas"}, inplace=True)
vinculo.rename(columns={"Mes Vinculo":"Mes", "Data Vínculo":"Contas Vinculadas"}, inplace=True)

In [45]:
df = abertura.merge(vinculo, on=['Assessor', 'Mes', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Faixa Cliente Performance'], how='outer')

In [46]:
# 1. Encontra a data mínima e máxima dentro da sua base tratada
# Usamos 'pd.to_datetime' para garantir que não dê erro de tipo
data_min = min(pd.to_datetime(base_filtrada['Data de Abertura']).min(), pd.to_datetime(base_filtrada['Data Vínculo']).min())
data_max = max(pd.to_datetime(base_filtrada['Data de Abertura']).max(), pd.to_datetime(base_filtrada['Data Vínculo']).max())

# 2. Cria a lista de datas mensais (frequência 'MS' = Month Start)
datas = pd.date_range(start=data_min, end=data_max, freq='MS')

print(f"Gerando calendário de {data_min.strftime('%m/%Y')} até {data_max.strftime('%m/%Y')}")

# --- SEU CÓDIGO ORIGINAL CONTINUA AQUI ---

df_datas_list = []

# Pegamos os valores únicos de cada coluna para criar a matriz completa
lista_assessores = df['Assessor'].unique() if 'Assessor' in df.columns else []
lista_faixas_abertura = df['Faixa Cliente Abertura'].unique() if 'Faixa Cliente Abertura' in df.columns else []
lista_faixas_atual = df['Faixa Cliente Atual'].unique() if 'Faixa Cliente Atual' in df.columns else []

# Agora o 'datas' existe e o loop vai funcionar
for data in datas:
    mes_str = data.strftime("%Y-%m")
    for assessor in lista_assessores:
        for faixa_abertura in lista_faixas_abertura:
            for faixa_atual in lista_faixas_atual:
                
                df_datas_list.append({
                    "Mes": mes_str,
                    "Assessor": assessor,
                    "Faixa Cliente Abertura": faixa_abertura,
                    "Faixa Cliente Atual": faixa_atual
                })

df_datas = pd.DataFrame(df_datas_list)

print(f"✅ Sucesso! Geradas {len(df_datas)} linhas de base para o histórico.")

Gerando calendário de 03/2016 até 01/2026
✅ Sucesso! Geradas 32096 linhas de base para o histórico.


In [47]:
# %%
df_datas_list = []

# Pegamos os valores únicos de cada coluna para criar a matriz completa
# Usamos .unique() e tratamos caso a coluna não exista para o script não parar
lista_assessores = df['Assessor'].unique() if 'Assessor' in df.columns else []
lista_faixas_abertura = df['Faixa Cliente Abertura'].unique() if 'Faixa Cliente Abertura' in df.columns else []
lista_faixas_atual = df['Faixa Cliente Atual'].unique() if 'Faixa Cliente Atual' in df.columns else []

for data in datas:
    mes_str = data.strftime("%Y-%m")
    for assessor in lista_assessores:
        for faixa_abertura in lista_faixas_abertura:
            for faixa_atual in lista_faixas_atual:
                # Criamos um dicionário simples para cada linha
                df_datas_list.append({
                    "Mes": mes_str,
                    "Assessor": assessor,
                    "Faixa Cliente Abertura": faixa_abertura,
                    "Faixa Cliente Atual": faixa_atual
                })

# Transforma a lista de dicionários em DataFrame de uma vez só
# Isso substitui o pd.concat(dfs) e já cria a coluna 'Mes'
df_datas = pd.DataFrame(df_datas_list)

print(f"✅ Sucesso! Geradas {len(df_datas)} linhas de base para o histórico.")

✅ Sucesso! Geradas 32096 linhas de base para o histórico.


In [48]:

colunas_necessarias = ["Assessor", "Mes", "Faixa Cliente Abertura", "Faixa Cliente Atual"]
colunas_atuais = df.columns.tolist()

missing = [c for c in colunas_necessarias if c not in colunas_atuais]

if not missing:
    print("Todas as colunas necessárias estão presentes!")
else:
    print(f"Erro: Faltam as seguintes colunas: {missing}")
    print(f"Colunas disponíveis: {colunas_atuais}")

Todas as colunas necessárias estão presentes!


In [49]:
# %%
# Realiza o merge para trazer as contagens de contas
df_final = pd.merge(
    df_datas, 
    df[['Mes', 'Assessor', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Contas Abertas', 'Contas Vinculadas']], 
    on=['Mes', 'Assessor', 'Faixa Cliente Abertura', 'Faixa Cliente Atual'], 
    how='left'
)

# Preenche os campos vazios com 0
df_final.fillna(0, inplace=True)

In [50]:
conexao = Connect.connect_techdb()
times = Connect.import_table(conexao, "times_nova_empresa")
conexao.close()

In [51]:
df_datas = df_datas[df_datas['Assessor'].isin(times['Assessor'])]

In [52]:
df = pd.merge(df_datas, df, on=['Mes', 'Faixa Cliente Abertura', 'Faixa Cliente Atual', 'Assessor'], how='outer')

In [53]:
df.fillna(0, inplace=True)

In [54]:
conexao = Connect.connect_techdb()
df.to_sql("abertura_de_conta", 
          con=conexao, 
          index=False, 
          if_exists='replace',
          schema='principal')
conexao.close()

In [55]:
contas_abertas = \
    base_filtrada[[
        'Data de Abertura', 'Data Vínculo', 'Conta', 'Nome',
        'Assessor', 'Tipo', 'PL Abertura', 'PL Atual'
    ]]

In [56]:
primeiro_dia_semana = (datetime.today() - timedelta(days = datetime.today().weekday())).strftime("%Y-%m-%d")

In [57]:
contas_abertas = contas_abertas[contas_abertas['Data Vínculo'] >= primeiro_dia_semana]

In [58]:
conexao =Connect.connect_techdb()
infos_clientes = Connect.import_table(conexao, "infos_clientes")
conexao.close()
infos_clientes.rename(columns={"Tipo":"Tipo de conta"}, inplace=True)

In [59]:
contas_abertas = contas_abertas.merge(infos_clientes[['Conta', 'Tipo de conta', 'Profissão / Setor', 'IDADE']], on='Conta', how='left')

In [60]:
contas_abertas.to_excel(r"C:\Scripts\backups_atria\diarios\arquivos_banco\scripts\relatorios\entradas_e_saidas\contas_novas_semana.xlsx", header=True, index=False)